# Acquiring and analysing building data

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mpmath as mp
import requests
import random

In [ ]:
%load-ext autoreload
% autreload 2

## Accessing the data

### Functions for converting to and from lon/lat to XYZ tile format

In [ ]:
def get_tile(lat_deg, lon_deg, zoom=15):
    """
    A function to get the relevant tile from lat,lon,zoom)
    """   
    lat_rad = mp.radians(lat_deg)
    n = 2 ** zoom
   
    xtile = n * ((lon_deg + 180) / 360)
    ytile = float(n * (1 - (mp.log(mp.tan(lat_rad) + mp.sec(lat_rad)) / np.pi)) / 2)
    return zoom, round(xtile), round(ytile) # 'tile %d/%d/%d '%

In [ ]:
def tile2lon(z,x,y):
    return x / 2**z * 360 - 180

def tile2lat(z,x,y):
    n = mp.pi - 2 * mp.pi * y / 2**z;
    return float((180 / mp.pi) * (mp.atan(0.5 * (mp.exp(n) - mp.exp(-n)))))

def tile_bbox(z,x,y):
    '''
    Returns the lat, lon bounding box of a tile
    '''
    w = tile2lon(z,x,y)
    s = tile2lat(z,x,y) 
    e = tile2lon(z,x+1,y)
    n = tile2lat(z,x,y+1)
    return [w,s,e,n]

## Function for calling the api

In [ ]:
def osmbuildings_request(latitude:float, longitude:float):
    """
    returns json response with building data from OSMBuildings
    API for a specific latitude and longitude
    """
    
    base_url = "https://data.osmbuildings.org/0.2/anonymous/tile"
    zoom, xtile, ytile = get_tile(latitude, longitude, 15)
    
    url = f"{base_url}/{zoom}/{xtile}/{ytile}.json"
    print(f"URL: {url}")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'}
    response = requests.get(url, headers = headers)
 
    print(f"Status code: {response.status_code}")
    json_response = response.json()

    return json_response

### Coordinates of cities

In [ ]:
paris_coords = {'upper_left': [48.813898, 2.264216],
                'lower_right': [48.900502, 2.42172]}

berlin_coords = {'upper_left': [52.475607, 13.313794],
                 'lower_right': [52.55571, 13.471299]}

london_coords = {'upper_left': [51.469149, -0.216408],
                 'lower_right': [51.551072, -0.058904]}

brussels_coords = {'upper_left': [50.808751, 4.288857],
                   'lower_right': [50.891855, 4.446361]}

city_bounds = {'paris': paris_coords,
               'berlin': berlin_coords,
               'london': london_coords,
               'brussels': brussels_coords}
cities = ['paris', 'berlin', 'london', 'brussels']

## Number of XYZ tiles for each

In [ ]:
def get_tiles(city_coords):
    """
    Gets the number of tiles that form the grid over the city
    """
    ul_tile = get_tile(lat_deg=city_coords['upper_left'][0], lon_deg=city_coords['upper_left'][1], zoom=15)
    lr_tile = get_tile(lat_deg=city_coords['lower_right'][0], lon_deg=city_coords['lower_right'][1], zoom=15)
    city_xtiles = np.abs(ul_tile[1] - lr_tile[1])
    city_ytiles = np.abs(ul_tile[2] - lr_tile[2])
    city_tiles = [city_xtiles, city_ytiles]
    return city_tiles

In [ ]:
tiles = {}
tiles['paris'] = get_tiles(paris_coords)
tiles['berlin'] = get_tiles(berlin_coords)
tiles['london'] = get_tiles(london_coords)
tiles['brussels'] = get_tiles(brussels_coords)
for city, tile_counts in tiles.items():
    message = f"The tiles covering {city.title()} are a grid of: {tile_counts}."
    print(message)

## Getting multiple tiles and checking materials documentation for each city

In [ ]:
def get_multiple_jsons(city_coords):
    """
    Gets jsons from a randomly seleted 10 xyz tiles in order to compare material documentation
    """
    jsons = []
    for i in range(10):
        random_lat = round(random.uniform(city_coords['upper_left'][0], city_coords['lower_right'][0]), 5)
        random_long = round(random.uniform(city_coords['upper_left'][1], city_coords['lower_right'][1]), 5)
        json_response = osmbuildings_request(random_lat, random_long)
        jsons.append(json_response)
    return jsons

In [ ]:
def get_materials(city_jsons):
    """
    Returns the proportion of buildings across all jsons for which the material was documented
    """
    materials = []
    total_features = 0
    for json_tile in city_jsons:
        total_features += len(json_tile['features'])
        for n in range(len(json_tile['features'])):
            if json_tile['features'][n]['properties'].get('material') != None:
                materials.append(json_tile['features'][n]['properties'].get('material'))
                proportion = len(materials)/len(json_tile['features'])
    proportion = len(materials)/total_features
    return proportion

In [ ]:
ouput_strings =[]
for city in cities:
    city_coords = city_bounds[city]
    city_jsons = get_multiple_jsons(city_coords)
    mat_prop = get_materials(city_jsons)
    ouput_strings.append(f"Proportion of materials in {city}: {mat_prop}")

In [ ]:
print(*ouput_strings, sep='\n')

### The level of documentation for building material is probably too low for any city (highest 0.05%)

## Getting all tiles covering a city, and the respective json files

In [ ]:
def get_all_tiles(city):
    """
    Returns two lists, one for each x and one for each y
    """
    starting_tile = []
    city_coords = city_bounds[city]
    city_tile_grid = tiles[city]
    starting_tile.append(get_tile(city_coords['upper_left'][0], city_coords['upper_left'][1])[1])
    starting_tile.append(get_tile(city_coords['upper_left'][0], city_coords['upper_left'][1])[2])
    x_tiles = []
    y_tiles = []
    for x in range(city_tile_grid[0]-1):
        x_tiles.append(starting_tile[0] + x)
    for y in range(city_tile_grid[1]-1):
        y_tiles.append(starting_tile[1] + y)
    return x_tiles, y_tiles

In [ ]:
def get_all_tile_jsons(x_tiles, y_tiles, zoom=15):
    """
    Returns jsons for all combinations of x and y within a given city
    """
    city_jsons = {}
    for x in x_tiles:
        for y in y_tiles:
            tile_lat = tile2lat(zoom, x, y)
            tile_lon = tile2lon(zoom, x, y)
            tile = (x, y)
            json_response = osmbuildings_request(tile_lat, tile_lon)
            city_jsons[tile] = json_response
    return city_jsons

In [ ]:
paris_x_tiles, paris_y_tiles = get_all_tiles('paris')
paris_jsons = get_all_tile_jsons(paris_x_tiles, paris_y_tiles)

In [ ]:
berlin_x_tiles, berlin_y_tiles = get_all_tiles('berlin')
berlin_jsons = get_all_tile_jsons(berlin_x_tiles, berlin_y_tiles)

In [ ]:
london_x_tiles, london_y_tiles = get_all_tiles('london')
london_jsons = get_all_tile_jsons(london_x_tiles, london_y_tiles)

In [ ]:
london_jsons.keys()

In [ ]:
london_jsons[(16364, 10902)]['features'][100]

In [ ]:
def get_roof_color(city_jsons):
    """
    Returns the proportion of buildings across all jsons for which the material was documented
    """
    roof_colors = []
    total_features = 0
    for json_tile in city_jsons.values():
        total_features += len(json_tile['features'])
        for n in range(len(json_tile['features'])):
            if json_tile['features'][n]['properties'].get('roofColor') != None:
                roof_colors.append(json_tile['features'][n]['properties'].get('roofColor'))
                proportion = len(roof_colors)/len(json_tile['features'])
    proportion = len(roof_colors)/total_features
    return proportion

In [ ]:
get_roof_color(london_jsons)

In [ ]:
def get_building_heights(city_jsons):
    heights = []
    for json in city_jsons.values():
        for building in json['features']:
            heights.append(building['properties']['height'])
    return heights

In [ ]:
heights = get_building_heights(london_jsons)

In [ ]:
berlin_heights = get_building_heights(berlin_jsons)
len(berlin_heights)

In [ ]:
paris_heights = get_building_heights(paris_jsons)
len(paris_heights)

In [ ]:
len(heights)

In [ ]:
plt.hist(heights);

In [ ]:
max = np.max(heights)

In [ ]:
max

In [ ]:
min = np.min(heights)

In [ ]:
min

In [ ]:
mean = np.mean(heights)

In [ ]:
mean

In [ ]:
coords = london_jsons[(16364, 10902)]['features'][5]['geometry']['coordinates'][0][:]
x = []
y = []
for coord in coords:
  x.append(coord[0])
  y.append(coord[1])

# london_jsons[(16364, 10904)]['features'][5]
# london_jsons[(16364, 10904)]['features'][10]

coords2 = london_jsons[(16364, 10902)]['features'][10]['geometry']['coordinates'][0][:]
x2 = []
y2 = []
for coord in coords2:
  x2.append(coord[0])
  y2.append(coord[1])

In [ ]:
print(london_jsons[(16364, 10902)]['features'][5])
print(london_jsons[(16364, 10902)]['features'][10])
print(london_jsons[(16364, 10902)]['features'][15])

In [ ]:
print(london_jsons[(16364, 10902)]['features'][100])

In [ ]:
print(london_jsons[(16364, 10904)]['features'][5])
print(london_jsons[(16364, 10904)]['features'][10])
print(london_jsons[(16364, 10904)]['features'][15])

In [ ]:
print(x, y, x2, y2, sep='\n')

In [ ]:
plt.plot(x, y)
plt.plot(x2, y2)

In [ ]:
# london_coords = {'upper_left': [51.469149, -0.216408],
#                  'lower_right': [51.551072, -0.058904]}

In [ ]:
def get_split_coords(building):
  coords = building['geometry']['coordinates'][0][:]
  x = []
  y = []
  for coord in coords:
    x.append(coord[0])
    y.append(coord[1])
  return x, y

In [ ]:
def plot_json(json):
  for building in json['features']:
    x, y = get_split_coords(building)
    plt.plot(x, y)

In [ ]:
def plot_whole_json_set(json_set):
  for json in json_set.values():
    plot_json(json)
  plt.show;

In [ ]:
testing_json = london_jsons[(16364, 10902)]

In [ ]:
plot_json(testing_json)

In [ ]:
fig = plt.figure(figsize=(35, 30))
plot_whole_json_set(london_jsons)

In [ ]:
fig = plt.figure(figsize=(35, 30))
plot_whole_json_set(berlin_jsons)

In [ ]:
fig = plt.figure(figsize=(35, 30))
plot_whole_json_set(paris_jsons)

In [ ]:
testing_json['features'][0]['geometry']['coordinates'][0]

In [ ]:
def receive_building_data(jsons):
  buildings = []
  for json in jsons.values():
    for building in json['features']:
      height = building['properties']['height']
      coords = building['geometry']['coordinates'][0]
      building_data = [height, coords]
      buildings.append(building_data)
  return buildings


In [ ]:
# def get_split_coords(building):
#   coords = building['geometry']['coordinates'][0][:]
#   x = []
#   y = []
#   for coord in coords:
#     x.append(coord[0])
#     y.append(coord[1])
#   return x, y

In [ ]:
# def get_height_lats_longs(jsons):
#   buildings = []
#   for json in jsons.values():
#     for building in json['features']:
#       x = []
#       y = []
#       for coord in building['geometry']['coordinates'][0][:]:
#         x.append(coord[0])
#         y.append(coord[1])
#       height = building['properties']['height']
#       coords = [x, y]
#       building_data = [height, coords]
#       buildings.append(building_data)
#   return buildings

In [ ]:
paris_buildings = receive_building_data(paris_jsons)

In [ ]:
len(paris_buildings)

In [ ]:
paris_buildings[:10]